In [1]:
import numpy as np
import pandas as pd
from shapely import geometry
import shapely.speedups
from tqdm import tqdm
from quetzal.model import stepmodel

In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none func

# Preparation of the transport network.
## Saves aggregated bus and short-distance rail network.
## Needs PT networks with access and egress.

In [2]:
input_path = '../input/'
output_path = '../output/'
model_path = '../model/'

In [3]:
# Loading StepModel with PT networks...
sm = stepmodel.read_json(model_path + 'de_pt_network')
bus = stepmodel.read_json(model_path + 'de_pt_network_bus')
# Loading access and egress
ae = stepmodel.read_json(model_path + 'de_pt_access_egress')

In [4]:
sm.nodes.sample()

,route_type,stop_name,FID,geometry
index,,,,
rail_long_node_3,rail_long_distance,Hanau Hbf,DE719_2,POINT (8.92900 50.12095)


In [5]:
# Check nodeset integrity for later steps to work
try:
    sm.integrity_test_nodeset_consistency()
except AssertionError:
    print('Found {} orphan nodes'.format(len(sm.orphan_nodes)))
    sm.nodes.drop(sm.orphan_nodes, inplace=True)
    # Test integrity again
    sm.integrity_test_nodeset_consistency()

Found 5 orphan nodes
no road_links or road_nodes


In [6]:
# Test sequences
# Use an own function because quetzal's takes ages
def test_sequences(trip):
    assert len(trip)==trip['link_sequence'].max(), \
        'broken sequence in trip {}'.format(trip['trip_id'].unique()[0])

In [7]:
# Fix sequences
# Use an own function because quetzal's takes ages
def fix_sequences(trip):
    trip = trip.sort_values('link_sequence')
    # Check link succession
    ind = list(trip.index)
    for i in range(len(trip.index) - 1):
        try:
            assert trip.loc[ind[i], 'b'] == trip.loc[ind[i+1], 'a'], \
                'broken trip {}: stop {} has no successor link'.format(
                    trip['trip_id'].unique()[0], trip.loc[ind[i], 'b'])
        except AssertionError:
            trip.loc[ind[i+1]:ind[-1], 'trip_id'] = \
                trip.loc[ind[i+1]:ind[-1], 'trip_id'] + '_' + str(i)
    # Repair sequences
    if len(trip) != trip['link_sequence'].max():
        trip['link_sequence'] = trip.groupby('trip_id')['link_sequence'].apply(
            lambda t: [j for j in range(1, len(t.index)+1)]).sum()
    return trip

In [8]:
# Test and save broken sequences
def test_sequences_save(trip):
    if len(trip)!=trip['link_sequence'].max():
        return list(trip.index)

In [9]:
tqdm.pandas()
try:
    sm.links.groupby('trip_id').progress_apply(test_sequences)
except AssertionError:
    links = sm.links.groupby('trip_id').progress_apply(fix_sequences).reset_index(level=0, drop=True)
    links.groupby('trip_id').progress_apply(test_sequences)
    sm.links = links

100%|██████████████████████████████████████████████████████████████████████████| 15701/15701 [00:05<00:00, 2847.26it/s]


In [10]:
broken_seqs = bus.links.groupby('trip_id').progress_apply(test_sequences_save)

100%|████████████████████████████████████████████████████████████████████████| 171343/171343 [00:56<00:00, 3024.23it/s]


In [11]:
broken_seqs.loc[broken_seqs.notna()]

trip_id
bus_1001610    [bus_2096605, bus_2096606, bus_2096607, bus_20...
bus_100624     [bus_606727, bus_606728, bus_606729, bus_60673...
bus_10068      [bus_113373, bus_113374, bus_113375, bus_11337...
bus_1007589    [bus_2102757, bus_2102758, bus_2102759, bus_21...
bus_1008462    [bus_2103833, bus_2103834, bus_2103835, bus_21...
                                     ...                        
bus_992064     [bus_2086772, bus_2086773, bus_2086774, bus_20...
bus_992371     [bus_2087008, bus_2087009, bus_2087010, bus_20...
bus_99326      [bus_602418, bus_602419, bus_602420, bus_60242...
bus_993782     [bus_2088311, bus_2088312, bus_2088313, bus_20...
bus_997461     [bus_2092137, bus_2092138, bus_2092139, bus_20...
Length: 1491, dtype: object

In [12]:
links = bus.links.loc[broken_seqs.loc[broken_seqs.notna()].sum()
                     ].groupby('trip_id').progress_apply(fix_sequences)

100%|██████████████████████████████████████████████████████████████████████████████| 1491/1491 [00:42<00:00, 35.17it/s]


In [13]:
links.sample()

,,route_id,route_type,a,b,time,trip_id,link_sequence,geometry
trip_id,index,,,,,,,,
bus_218292,bus_x_39533,bus_16481,bus,bus_n_x_8485,bus_n_x_346015,240.0,bus_218292_0_1_2_4_5_6_7_8_10_11_12,2,"LINESTRING (7.38869 50.07647, 7.36236 50.07692)"


In [14]:
links.shape

(41283, 8)

In [15]:
links.reset_index(level=0, drop=True, inplace=True)

In [16]:
links.groupby('trip_id').progress_apply(test_sequences)

100%|██████████████████████████████████████████████████████████████████████████| 23521/23521 [00:07<00:00, 3221.15it/s]


""


In [17]:
bus.links = bus.links.drop(broken_seqs.loc[broken_seqs.notna()].sum()).append(links)

In [18]:
bus.links.sample()

,route_id,route_type,a,b,time,trip_id,link_sequence,geometry
index,,,,,,,,
bus_324309,bus_15828,bus,bus_n_376058,bus_n_18637,120.0,bus_38432,30,"LINESTRING (13.78050 51.24906, 13.77373 51.25892)"


In [19]:
bus.nodes.sample()

,stop_name,route_type,FID,geometry
index,,,,
bus_n_259189,Hannover Hangstraße Haltestelle 0,bus,DE929_1,POINT (9.69165 52.33123)


In [20]:
# Drop nodes outside of zones
if 'FID' in sm.nodes.columns:
    sm.nodes = sm.nodes[sm.nodes['FID'].notna()]
    bus.nodes = bus.nodes[bus.nodes['FID'].notna()]
    print(sm.nodes.shape)
    print(bus.nodes.shape)

(14969, 4)
(402790, 4)


In [21]:
# Divide nodes
print(sm.nodes.shape)
disagg_nodes = sm.nodes.loc[sm.nodes['route_type']=='rail_short_distance'].append(bus.nodes)
sm.nodes = sm.nodes.loc[sm.nodes['route_type']!='rail_short_distance']
print(disagg_nodes.shape)

(14969, 4)
(416516, 4)


In [22]:
# Divide links
print(sm.links.shape)
disagg_links = sm.links.loc[sm.links['route_type']=='rail_short_distance'].append(bus.links)
sm.links = sm.links.loc[sm.links['route_type']!='rail_short_distance']
print(disagg_links.shape)

(146730, 8)
(2504642, 8)


In [23]:
# Number of trips
len(disagg_links['trip_id'].unique())

204977

## Remove unneccessary stops
Bus and short-distance rail service in the GTFS feeds contain trips with not further connected intermediate stops. Thus, the PT network graph can be reduced without loss of information.

In [24]:
sm.links.sample()

,a,b,link_sequence,route_id,route_type,time,trip_id,geometry
index,,,,,,,,
coach_1896,coach_node_FLIXBUS:3618,coach_node_FLIXBUS:8898,5.0,coach_FLIXBUS:025,coach,300.0,coach_FLIXBUS:025:30:14:00:00Z-6,"LINESTRING (13.16153 53.00407, 13.19219 53.03737)"


In [25]:
# Count the number of links to/from each node
n_links_dict = disagg_links[['a', 'b']].stack().value_counts().to_dict()
connector_set = ae.zone_to_transit[['a', 'b']].append(ae.footpaths[['a', 'b']])
n_connectors_dict = connector_set.stack().value_counts().to_dict()
disagg_nodes['n_links'] = [n_links_dict[i] for i in list(disagg_nodes.index)]
disagg_nodes['n_connectors'] = disagg_nodes.index.map(n_connectors_dict)
disagg_nodes['n_connectors'].replace(np.nan, 0, inplace=True)

In [26]:
disagg_nodes.loc[disagg_nodes.isna().any(axis=1)]

,FID,geometry,route_type,stop_name,n_links,n_connectors
index,,,,,,


In [27]:
print(disagg_nodes.loc[disagg_nodes['route_type']=='bus'].shape)
print(disagg_nodes.loc[disagg_nodes['route_type']!='bus'].shape)

(402790, 6)
(13726, 6)


In [28]:
disagg_nodes.sample(2)

,FID,geometry,route_type,stop_name,n_links,n_connectors
index,,,,,,
bus_n_11549,DE271,POINT (10.84875 48.39334),bus,"Augsburg, Am Eulenhorst",8,0.0
bus_n_124994,DE21M,POINT (12.37048 48.07121),bus,Schnaitsee Postamt,8,0.0


In [29]:
# Keep interconnected rail trips but drop all bus trips
# without connection to another mode or centroid
agg_nodes = disagg_nodes.loc[
    ((disagg_nodes['route_type']!='bus') &
     ((disagg_nodes['n_links'] > 2) |
      (disagg_nodes['n_connectors'] > 0))
    ) | (
        (disagg_nodes['route_type']=='bus') &
        (disagg_nodes['n_connectors'] > 0)
    )]

In [30]:
print(agg_nodes.loc[agg_nodes['route_type']=='bus'].shape)
print(agg_nodes.loc[agg_nodes['route_type']!='bus'].shape)

(38946, 6)
(13351, 6)


### Aggregate links within the broken trips

In [31]:
# Mark dropped nodes
disagg_links['relevant'] = disagg_links['a'].isin(list(agg_nodes.index)) | \
    disagg_links['b'].isin(list(agg_nodes.index))

In [32]:
disagg_links.loc[disagg_links['relevant']].shape

(579625, 9)

In [33]:
disagg_links.sample()

,a,b,geometry,link_sequence,route_id,route_type,time,trip_id,relevant
index,,,,,,,,,
bus_776609,bus_n_203522,bus_n_70745,"LINESTRING (9.96777 53.26711, 9.95661 53.26929)",9.0,bus_8987,bus,60.0,bus_154154,False


In [34]:
# Function for aggregating links
def agg_trips(trip):
    # Drop links with missing nodes
    trip_agg = trip.loc[trip['relevant']].sort_values('link_sequence')
    if len(trip_agg.index) == 0:
        # Trip is fully irrelevant
        return
    
    missing_nodes = list(set(list(trip_agg['a'])+list(trip_agg['b'])) -
                         set(agg_nodes.index))
    if len(missing_nodes)==0 and len(trip.index)==len(trip_agg.index):
        # This trip is not affected
        return trip
    
    # Repair link succession
    ind = list(trip_agg.index)
    for i in range(len(ind) - 1):
        if trip_agg.loc[ind[i], 'b'] in missing_nodes:
            trip_agg.loc[ind[i + 1], 'a'] = trip_agg.loc[ind[i], 'a']
            try:
                trip_agg.loc[ind[i + 1], 'geometry'] = geometry.LineString(
                    [agg_nodes.loc[trip_agg.loc[ind[i + 1], 'a'], 'geometry'],
                     agg_nodes.loc[trip_agg.loc[ind[i + 1], 'b'], 'geometry']])
            except KeyError:
                return
            trip_agg.drop(ind[i], inplace=True)
            i = i + 1
    
    ind = list(trip_agg.index)
    if len(trip_agg.index) > 0 and trip_agg.loc[ind[0], 'a'] in missing_nodes:
        # Drop unused first link
        trip_agg = trip_agg.iloc[1:]
    if len(trip_agg.index) > 0 and trip_agg.loc[ind[-1], 'b'] in missing_nodes:
        # Drop unused last link
        trip_agg = trip_agg.iloc[:-1]
    ind = list(trip_agg.index)
    if len(ind) == 0:
        return
    
    # Aggregate travel time
    for i in range(len(ind) - 1):
        try:
            assert trip_agg.loc[ind[i], 'b'] == trip_agg.loc[ind[i+1], 'a'], \
                'broken sequence in trip {}: stop {} has no successor link'.format(
                    trip_agg['trip_id'].unique()[0], trip_agg.loc[ind[i], 'b'])
        except AssertionError:
            # Drop this trip
            return
        if trip_agg.loc[ind[i + 1], 'link_sequence'] - trip_agg.loc[ind[i], 'link_sequence'] > 1:
            trip_agg.loc[ind[i], 'time'] = trip.loc[ind[i]:ind[i+1], 'time'].sum() - \
                trip_agg.loc[ind[i+1], 'time'] # pandas slicing includes both boundaries
    
    # Reindex the sequence numbers
    trip_agg['link_sequence'] = [i for i in range(1, len(trip_agg.index)+1)]
    
    return trip_agg

In [35]:
# Faster variant for using multiple cores
'''import multiprocessing as mp
with mp.Pool(processes=10) as p:
    agg_links = pd.concat(p.map(
        agg_trips, [g for _, g in disagg_links.groupby('trip_id')]))'''

"import multiprocessing as mp\nwith mp.Pool(processes=10) as p:\n    agg_links = pd.concat(p.map(\n        agg_trips, [g for _, g in disagg_links.groupby('trip_id')]))"

In [36]:
tqdm.pandas()
agg_links = disagg_links.groupby('trip_id').progress_apply(
    agg_trips).reset_index(level=0, drop=True)

100%|██████████████████████████████████████████████████████████████████████████| 204977/204977 [36:06<00:00, 94.60it/s]


In [37]:
agg_links.drop('relevant', axis=1, inplace=True)
agg_links.shape

(365249, 8)

In [38]:
agg_links.loc[~(agg_links['a'].isin(list(agg_nodes.index))) |
    ~(agg_links['b'].isin(list(agg_nodes.index)))]

,a,b,geometry,link_sequence,route_id,route_type,time,trip_id
index,,,,,,,,
bus_x_94358,bus_n_x_158628,bus_n_x_176123,"LINESTRING (14.06439 53.67318, 14.07166 53.68113)",1.0,bus_14417,bus,60.0,bus_1030749
bus_630184,bus_n_284216,bus_n_362496,"LINESTRING (9.81533 48.62884, 9.82044 48.62743)",1.0,bus_17239,bus,120.0,bus_107210
bus_x_24170,bus_n_x_158628,bus_n_x_176123,"LINESTRING (14.06439 53.67318, 14.07166 53.68113)",1.0,bus_14573,bus,60.0,bus_107865
bus_2198898,bus_n_225734,bus_n_116236,"LINESTRING (7.71711 47.95744, 7.75401 47.95625)",1.0,bus_16726,bus,240.0,bus_1100444
bus_123637,bus_n_245085,bus_n_168072,"LINESTRING (13.37043 50.86801, 13.38087 50.86875)",1.0,bus_12111,bus,120.0,bus_11051
bus_2264273,bus_n_33455,bus_n_230947,"LINESTRING (11.21028 53.41921, 11.21446 53.41659)",1.0,bus_18233,bus,60.0,bus_1167445
bus_165914,bus_n_283974,bus_n_252553,"LINESTRING (9.17067 53.68803, 9.16603 53.68442)",1.0,bus_15955,bus,300.0,bus_15656
bus_817275,bus_n_325822,bus_n_325591,"LINESTRING (12.83847 54.20343, 12.82188 54.20661)",1.0,bus_4388,bus,120.0,bus_169264
bus_917594,bus_n_181487,bus_n_160673,"LINESTRING (12.59083 51.27147, 12.58931 51.27475)",1.0,bus_16980,bus,120.0,bus_208398


In [39]:
# Drop these erronous trips
ids = agg_links.loc[~(agg_links['a'].isin(list(agg_nodes.index))) |
                    ~(agg_links['b'].isin(list(agg_nodes.index)))].index
agg_links = agg_links.loc[~agg_links['trip_id'].isin(list(agg_links.loc[ids, 'trip_id']))]

### Merge aggregated links and nodes with the model

In [40]:
sm.links = sm.links.loc[sm.links['route_type'].isin(
    ['bus', 'rail_short_distance'])==False]
sm.nodes = sm.nodes.loc[sm.nodes['route_type'].isin(
    ['bus', 'rail_short_distance'])==False]

In [41]:
# Re-add links to model
sm.links = sm.links.append(agg_links)
sm.links.shape

(384918, 8)

In [42]:
# Re-add nodes to the model
sm.nodes = sm.nodes.append(agg_nodes)
sm.nodes.shape

(53540, 6)

In [43]:
try:
    sm.integrity_test_nodeset_consistency()
except AssertionError:
    print('Number of orphan nodes: {}'.format(
        len(sm.orphan_nodes)))
    print('Number of missing nodes: {}'.format(
        len(sm.missing_nodes)))

Number of orphan nodes: 0
Number of missing nodes: 2


In [44]:
# Leave these nodes if they connect by more than one
# footpath or access/egress link
assert len(agg_nodes.loc[sm.orphan_nodes].loc[agg_nodes['n_connectors']>1].index) == len(sm.orphan_nodes)

In [45]:
sm.nodes.drop(['n_links', 'n_connectors'], axis=1, inplace=True)

In [46]:
sm.links.groupby('trip_id').progress_apply(test_sequences)

100%|████████████████████████████████████████████████████████████████████████| 101070/101070 [00:33<00:00, 3024.24it/s]


""


## Map nodes to zones
Needed for the accessibility calculation

In [47]:
# Nodes must be a GeoDataFrame
if 'FID' not in sm.nodes.columns:
    import geopandas as gpd
    sm.nodes = gpd.GeoDataFrame(sm.nodes, crs=sm.epsg)
    shapely.speedups.enable()
    sm.nodes['FID'] = np.nan
    for _, zone in tqdm(sm.zones.iterrows(), total=sm.zones.shape[0]):
        sm.nodes.loc[sm.nodes['geometry'].within(zone['geometry']), 'FID'] = zone['FID']

In [48]:
sm.nodes[sm.nodes['FID'].isna()]

,FID,geometry,route_type,stop_name
index,,,,


## Save model


In [49]:
# Add bus service to ancilliary
sm.agencies = sm.agencies.append(bus.agencies)
sm.pt_routes = sm.pt_routes.append(bus.pt_routes)

In [50]:
# Now, we have bus services in the same tables
sm.pt_route_types.append('bus')

In [51]:
# Split links in graph and auxiliary information
# for file sizes being compatible with github's size limit
cols = ['link_sequence', 'route_id', 'time', 'trip_id']
auxiliary = sm.links[cols]
sm.links.drop(cols, axis=1, inplace=True)

In [52]:
# Saving model...
sm.to_json(model_path + 'de_pt_network_agg',
           only_attributes=['zones', 'links', 'nodes', 'pt_route_types'],
           encoding='utf-8')
sm.to_json(model_path + 'de_pt_network_ancillary',
           only_attributes=['agencies', 'pt_routes'],
           encoding='utf-8')

to_hdf(overwriting): 100%|████████████████████████████████████████████████████████████| 38/38 [00:00<00:00, 140.79it/s]


In [53]:
# Save auxiliary information seperately
auxiliary['index'] = auxiliary.index
auxiliary.reset_index(drop=True, inplace=True)
auxiliary.to_json(model_path + 'de_pt_network_agg/links_quetzaldata.json')